# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Цель проекта: необходимо решить, какой из 3 регионов взять в разработку, с учетом экономической эффективности в представленном нефтедобывающем регионе и риска получения убытков, т е, по-сути, просчитать экономический эффект отдельно для каждого региона.


## Загрузка и подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from numpy.random import RandomState
from scipy import stats as st
pd.options.display.float_format = '{:.2f}'.format


In [ ]:
try:
    df_0 = pd.read_csv('/datasets/geo_data_0.csv')
    df_1 = pd.read_csv('/datasets/geo_data_1.csv')
    df_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    df_0 = pd.read_csv('geo_data_0.csv')
    df_1 = pd.read_csv('geo_data_1.csv')
    df_2 = pd.read_csv('geo_data_2.csv')

для начала, изучим основную информацию по датасетам: у нас их 3. Также выведем на экран первые строки датасетов, для изучения и визуальной оценки данных.

In [ ]:
df_0.head(5)

In [ ]:
df_0.info()

In [ ]:
df_1.head(5)

In [ ]:
df_1.info()

In [ ]:
df_2.head(5)

In [ ]:
df_2.info()

In [ ]:
df_0.nunique()

In [ ]:
df_1.nunique()

In [ ]:
df_2.info()

In [ ]:
df_1['product'].value_counts()

на основании изучения основной информации по датасетам и таблицам данных, делаем вывод о том, что типы данных соответствуют признакам, пропуски отсутствуют. Колонка id не несет информационной ценности - ее мы удалим. В df_1 в колонке product только 12 уникальных значений. Датасеты можно принимать в работу.

In [ ]:
df_0 = df_0.drop('id', axis = 1)
df_1 = df_1.drop('id', axis = 1)
df_2 = df_2.drop('id', axis = 1)

In [ ]:
df_0.info()

In [ ]:
df_1.info()

In [ ]:
df_2.info()

Все в порядке: id  в датасетах удалены. Датасеты можно принимать в работу.

In [ ]:
df_0.describe()

In [ ]:
df_0.corr()

In [ ]:
df_1.describe()

In [ ]:
df_1.corr()

In [ ]:
df_2.describe()

In [ ]:
df_2.corr()

## Обучение и проверка модели

разбиваем данные по 3 датафреймам на обучающую и валидационную выборки

In [ ]:
df_0_train, df_0_valid = train_test_split(df_0, test_size=0.25, random_state=12345)
df_1_train, df_1_valid = train_test_split(df_1, test_size=0.25, random_state=12345)
df_2_train, df_2_valid = train_test_split(df_2, test_size=0.25, random_state=12345)

сразу верифицируем чистоту разбиения на выборки

In [ ]:
df_0_valid.shape

In [ ]:
df_1_valid.shape

In [ ]:
df_2_valid.shape

теперь проведем масштабирование признаков, с целью стандартизации

In [ ]:
pd.options.mode.chained_assignment = None
numeric = ['f0']
scaler = StandardScaler()
scaler.fit(df_0_train[numeric]) 
df_0_train[numeric] = scaler.transform(df_0_train[numeric])
df_0_valid[numeric] = scaler.transform(df_0_valid[numeric])

In [ ]:
pd.options.mode.chained_assignment = None
numeric = ['f1']
scaler = StandardScaler()
scaler.fit(df_1_train[numeric]) 
df_1_train[numeric] = scaler.transform(df_1_train[numeric])
df_1_valid[numeric] = scaler.transform(df_1_valid[numeric])

In [ ]:
pd.options.mode.chained_assignment = None
numeric = ['f2']
scaler = StandardScaler()
scaler.fit(df_0_train[numeric]) 
df_2_train[numeric] = scaler.transform(df_2_train[numeric])
df_2_valid[numeric] = scaler.transform(df_2_valid[numeric])

все в порядке: разбиение прошло правильно. Двигаемся далее. Выделяем целевой признак для тренинговой и валидационной выборки по каждому датасету отдельно.

In [ ]:
target_train_0 = df_0_train['product']
features_train_0 = df_0_train.drop('product', axis = 1)
target_valid_0 = df_0_valid['product']
features_valid_0 = df_0_valid.drop('product', axis = 1)

target_train_1 = df_1_train['product']
features_train_1 = df_1_train.drop('product', axis = 1)
target_valid_1 = df_1_valid['product']
features_valid_1 = df_1_valid.drop('product', axis = 1)

target_train_2 = df_2_train['product']
features_train_2 = df_2_train.drop('product', axis = 1)
target_valid_2 = df_2_valid['product']
features_valid_2 = df_2_valid.drop('product', axis = 1)

проводим проверку по каждой полученной выборке (выполнил проверку по всем выборкам отдельно для каждого датасета, сохранил в коде только часть, т к все нормально).

In [ ]:
features_train_2.shape

In [ ]:
features_valid_2.shape

переходим к обучению модели линейной регрессии для каждого датасета отдельно. По результату, выведем на экран предсказанные значения, а также сравним их с средними фактическими значениями

In [ ]:
model_0 = LinearRegression().fit(features_train_0, target_train_0)
predicted_valid_0 = model_0.predict(features_valid_0)
mse = mean_squared_error (target_valid_0, predicted_valid_0)
print('MSE = ', mse)
print('RMSE = ', mse ** 0.5)
print("R2 = ", r2_score(target_valid_0, predicted_valid_0))
print('mae = ', mean_absolute_error(target_valid_0, predicted_valid_0))

In [ ]:
predicted_valid_00 = pd.Series(predicted_valid_0)
target_valid_00 = pd.Series(target_valid_0)
target_valid_00 = target_valid_0.reset_index(drop=True)

In [ ]:
model_1 = LinearRegression().fit(features_train_1, target_train_1)
predicted_valid_1 = model_1.predict(features_valid_1)
mse = mean_squared_error (target_valid_1, predicted_valid_1)
print('MSE = ', mse)
print('RMSE = ', mse ** 0.5)
print("R2 =", r2_score(target_valid_1, predicted_valid_1))
print('mae = ', mean_absolute_error(target_valid_1, predicted_valid_1))

In [ ]:
predicted_valid_11 = pd.Series(predicted_valid_1)
target_valid_11 = pd.Series(target_valid_1)
target_valid_11 = target_valid_1.reset_index(drop=True)

In [ ]:
model_2 = LinearRegression().fit(features_train_2, target_train_2)
predicted_valid_2 = model_2.predict(features_valid_2)
mse = mean_squared_error (target_valid_2, predicted_valid_2)
print('MSE = ', mse)
print('RMSE = ', mse ** 0.5)
print("R2 =", r2_score(target_valid_2, predicted_valid_2))
print('mae = ', mean_absolute_error(target_valid_2, predicted_valid_2))

In [ ]:
predicted_valid_22 = pd.Series(predicted_valid_2)
target_valid_22 = pd.Series(target_valid_2)
target_valid_22 = target_valid_2.reset_index(drop=True)

Вывод по итогам части 2: Нами проведена подготовка данных к построению предсказательной модели по каждому из 3 регионов. Проведено обучение на тренировочных выборках, получен  результат, показывающий, что лучшая предсказательная модель по метрикам - модель для второго региона.

In [ ]:
def series_type(target, pred):
    target = pd.Series(target)
    target_base = target.reset_index(drop = True)
    pred = pd.Series(pred)
    return target_base, pred

In [ ]:
series_type(target_valid_0, predicted_valid_0)
series_type(target_valid_1, predicted_valid_1)
series_type(target_valid_2, predicted_valid_2)

## Подготовка к расчёту прибыли

задаем константы из условия задачи

In [ ]:
exploration_spots = 500 # при разведке региона исследуют 500 точек
chosen_spots = 200 # из которых с помощью машинного обучения выбирают 200 лучших для разработки
technology_budget = 10**10 # бюджет на разработку скважин в регионе — 10 млрд рублей
income_per_barrel = 450000 # доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей

исходя из условий задачи, расчитаем достаточный объем сырья, необходимый для разработки новой скважины

In [ ]:
break_even = technology_budget/(income_per_barrel * 200)

In [ ]:
break_even

 баррелей нефти из новой скважины окупают эту скважину, т е это точка безубыточности для скважиныю Построим таблицу, в которую сведем всю полученную на данном этапе информацию

In [ ]:
gb_0_region = target_valid_00.mean() * income_per_barrel/1000
gb_1_region = target_valid_11.mean() * income_per_barrel/1000
gb_2_region = target_valid_22.mean() * income_per_barrel/1000

In [ ]:
gb = pd.DataFrame({'выручка млн. руб.\ скважину': [gb_0_region, gb_1_region, gb_2_region],\
                   'предсказаный дебет, тыс.бар': [predicted_valid_0.mean(), predicted_valid_1.mean(), predicted_valid_2.mean()],\
                   'фактический дебет, тыс. бар': [target_valid_0.mean(), target_valid_1.mean(), target_valid_2.mean()],\
                  'RMSE': ['37.57', '0.89', '40.02'], 'R2': ['0.28', '0.1', '0.20'], 'mae': ['30.92', '0.71', '32.79']})

gb.index = ['Регион 1', 'Регион 2', 'Регион 3']

In [ ]:
gb.round(2)

Вывод: Для достижения точки безубыточности необходимый объем добываемой нефти должен составлять не менее 111.11 тыс. баррелей на нефтедобывающий регион. На данном этапе лучшие предсказательные метрики в Регионе 2. В тоже самое время, требуется внимательный поиск региона для бурения, поскольку среднее содержание запасов меньше точки безубыточности.

## Расчёт прибыли и рисков 

Построим функцию расчета прибыли

In [ ]:
def sum_profit(target, predicted):
    predicted_sorted = predicted.sort_values(ascending = False)
    selected = target[predicted_sorted.index][:chosen_spots] # отбираем по predict, а суммируем по target !!
    #selected = selected.reset_index(drop=True) #сбрасываю индексы
    product_sum_max2000 = math.floor((selected.sum() * income_per_barrel)- technology_budget)
    return product_sum_max2000

In [ ]:
revenue_1 = sum_profit(target_valid_00, predicted_valid_00)
revenue_2 = sum_profit(target_valid_11, predicted_valid_11)
revenue_3 = sum_profit(target_valid_22, predicted_valid_22)

In [ ]:
#revenue_1 = sum_profit(target_valid_00.reset_index(drop = True), predicted_valid_00.reset_index(drop = True))
#revenue_2 = sum_profit(target_valid_11.reset_index(drop = True), predicted_valid_11.reset_index(drop = True))
#revenue_3 = sum_profit(target_valid_22.reset_index(drop = True), predicted_valid_22.reset_index(drop = True))

In [ ]:
print(f'Прибыль для полученного объёма баррелей нефти в первом регионе: {revenue_1} руб.' )
print(f'Прибыль для полученного объёма баррелей нефти во втором регионе: {revenue_2} руб.' )
print(f'Прибыль для полученного объёма баррелей нефти в третьем регионе: {revenue_3} руб.' )

Вывод: проведен расчет прибыли для полученного объема сырья в 3 регионах. Наибольшая прибыль на данном этапе расчитана для первого региона. Но для окончательного решения этого мало: необходимо расчитать риски. 


проводим процедуру Bootstrap, c целью последущего обсчета доверительного интервала и вычисления рисков

In [ ]:
def bootstrap(target_valid_region, predicted_valid_region):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        semi_sample =target_valid_region.sample(n=exploration_spots,replace=True,random_state=state)
        probs_subsample = predicted_valid_region[semi_sample.index]#отбираем предсказанные значения, для 500 реальных по индексу
        semi_sample.reset_index(drop=True, inplace = True) #сбрасываю индексы, + ..inplace = True
        probs_subsample.reset_index(drop=True, inplace = True) #сбрасываю индексы, + ..inplace = True
        print(semi_sample)
        print(probs_subsample)
        values.append(sum_profit(semi_sample, probs_subsample))
    values = pd.Series(values)
    
    confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), np.std(values, ddof=1))
    lower = int(values.quantile(0.025))
    higher = int(values.quantile(0.975))
    risk = (values < 0).mean()
    mean = values.mean()
    loss = values[values<0]
    final_score = 1. * loss.count()/len(values)
    
    loss_probability_min = 0.025
    print("Средняя прибыль: {:.2f} млн.руб.".format(mean))
    print("95%-й доверительный интервал: от {:.2f}".format(lower), "млн.руб. до {:.2f}".format(higher), "млн.руб.")
    print("Количество убыточных экспериментов:", loss.count())
    if final_score < loss_probability_min:
        print("Вероятность убытков равна {:.2%} и является меньше допустимой, регион подходит по критериям".format(final_score))
    else:
        print("Вероятность убытков равна {:.2%} и является больше допустимой, регион не подходит по критериям".format(final_score))
        print("Максимальный убыток: {:.2f} млн.руб".format(loss.min()))
        print("Максимальная прибыль: {:.2f} млрд.руб".format(values.max()))
    

    plt.figure()
    profit_plot = plt.hist(values, bins=100)
    one_x12, one_y12 = [confidence_interval[0],confidence_interval[0]], [0, 30]
    two_x12, two_y12 = [confidence_interval[1],confidence_interval[1]], [0, 30]
    plt.title('Гистограмма распределения прибыли в выбранном регионе')
    plt.xlabel('Прибыль в млн. рублей')
    plt.plot(one_x12, one_y12, two_x12, two_y12, marker = 'o')
    plt.show()

извлечем значения функции, ниже сведем их в таблицу

In [ ]:
bootstrap(target_valid_00, predicted_valid_00)

In [ ]:
bootstrap(target_valid_11, predicted_valid_11)

In [ ]:
bootstrap(target_valid_22, predicted_valid_22)

In [ ]:
rev_global = pd.DataFrame({'прибыль, руб': [revenue_1, revenue_2, revenue_3],\
                           'среднее значение прибыли, руб ': [425938526.40,  515222772.94, 435008362.28],\
                           'quantile 0.025' : [-102090094, 68873225, -128880547],\
                           'quantile 0.0975' : [947976352, 931547590, 969706953],\
                           'риск %' : [6, 1, 6.4]
                           })
rev_global.index = ['Регион 1', 'Регион 2', 'Регион 3']

In [ ]:
rev_global

<h3> Вывод: <a class="tocSkip"> </h3> <b>По итогу исследования 3 нефтяных регионов, построено 3 гистограммы, на которых показан вычисленный доверительный интервал. Нами вычислены показатели:</b>  Средняя прибыль, 95%-й доверительный интервал, Количество убыточных экспериментов, Вероятность убытков, Максимальный убыток, Максимальная прибыль. Проведен комплексный анализ перечисленных показателей. <b>С учетом окупаемости региона и рисков получения убытков, целесообразно остановиться на выборе второго региона, т к здесь риск получения убытков существенно ниже, чем в регионах 1 и 3.<b>